In [ ]:
import shutil
!pip install pygit2==1.12.2
%cd /content
folder_path = "/content/PlaylistCreator"
shutil.rmtree(folder_path, ignore_errors=True)
!git clone https://github.com/thefffilo/PlaylistCreator.git
%cd /content/PlaylistCreator/frontend
!npm i
!npm run build

In [ ]:
from google.colab import output
output.serve_kernel_port_as_window(8084)

## Import Max

### Imports

In [ ]:
# !pip install git+https://github.com/huggingface/datasets.git
# !pip install git+https://github.com/huggingface/transformers.git
!pip install torchaudio
!pip install librosa

In [ ]:
import torch
from transformers import AutoConfig, Wav2Vec2FeatureExtractor, HubertForSequenceClassification
import torchaudio
import torch.nn.functional as F

import librosa
import IPython.display as ipd
import numpy as np
import pandas as pd

### Define model

In [ ]:

device2 = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name2 = "pedromatias97/genre-recognizer-finetuned-gtzan_dset"
config2 = AutoConfig.from_pretrained(model_name2)
feature_extractor2 = Wav2Vec2FeatureExtractor.from_pretrained(model_name2)
model2 = HubertForSequenceClassification.from_pretrained(model_name2).to(device2)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/94.8M [00:00<?, ?B/s]

Some weights of the model checkpoint at pedromatias97/genre-recognizer-finetuned-gtzan_dset were not used when initializing HubertForSequenceClassification: ['hubert.encoder.pos_conv_embed.conv.weight_g', 'hubert.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at pedromatias97/genre-recognizer-finetuned-gtzan_dset and are newly initialized: ['hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original0', '

In [ ]:
def audio_file_to_input_array(audio_file, feature_extractor):
    # Carica il file audio
    speech_array, _ = librosa.load(audio_file, sr=feature_extractor.sampling_rate, mono=True)
    # Estrai le caratteristiche audio utilizzando l'estrattore appropriato
    inputs = feature_extractor(speech_array, return_tensors="pt", padding=True)
    return inputs

def predict(inputs, model, device):
    inputs = {key: val.to(device) for key, val in inputs.items()}
    # Esegui l'inferenza sul modello
    with torch.no_grad():
        logits = model(**inputs).logits

    # Calcola le probabilità e le etichette previste
    scores = F.softmax(logits, dim=1).detach().cpu().numpy()[0]
    outputs = [{"Label": i, "Score": f"{round(score * 100, 3):.1f}%"} for i, score in enumerate(scores)]
    return outputs

## Flask

In [ ]:
from flask import Flask, send_from_directory, jsonify, request
import os
# from spotipy.oauth2 import SpotifyOAuth
import random

import requests
from requests.auth import HTTPBasicAuth

app = Flask(__name__, static_folder='/content/PlaylistCreator/frontend/build', static_url_path='')
#app = Flask(__name__)
# run_with_ngrok(app)

genre_id= ["blues","classical","country","disco","hiphop","jazz","metal","pop","reggae","rock"]
@app.route('/uploadSong', methods=["POST"])
def uploadSong():
    # Assicurati che il file sia presente nella richiesta
    if 'audio' not in request.files:
        return jsonify({"error": "No audio file part"}), 400

    audio_file = request.files['audio']

    # Verifica se il file è stato caricato correttamente
    if audio_file.filename == '':
        return jsonify({"error": "No selected audio file"}), 400

    # Converte il file audio in un tensore di input utilizzando l'estrattore di caratteristiche
    inputs = audio_file_to_input_array(audio_file, feature_extractor2)

    # Esegui l'inferenza sul modello
    outputs = predict(inputs, model2, device2)
    print(outputs)
    # Trova il massimo punteggio e il relativo indice del genere
    max_score = 0
    max_genre_index = None
    for output in outputs:
        score = float(output['Score'].replace('%', ''))
        if score > max_score:
            max_score = score
            max_genre_index = output['Label']

    # Ritorna il massimo punteggio e il relativo indice del genere
    return jsonify({"genre": genre_id[max_genre_index]})

@app.route('/')
def serve():
    return send_from_directory(app.static_folder, 'index.html')

@app.route('/<path:path>')
def static_proxy(path):
    file_name = path.split('/')[-1]
    directory_name = os.path.join(app.static_folder, '/'.join(path.split('/')[:-1]))
    return send_from_directory(directory_name, file_name)


if __name__ == '__main__':
  app.run(host='0.0.0.0', port=8084)